In [61]:
import modules.resnet_3d as resnet
from pathlib import Path
import numpy as np
from modules.mapper import FileMapper as fm
from modules.dataset import Dataset
from modules.dataset_image_only import Dataset as dataset_2
from modules.cnn_model import CNNModel
from torch.utils.data import DataLoader
import torchtuples as tt
from pycox.models import LogisticHazard
import pickle
import random
from modules.mlflow_logger import Logger
from modules.utils import getPatientIdentifier, collate_fn, returnMontage, get_optimizer, pad_image
import sys
import argparse
import datetime
from pycox.evaluation import EvalSurv
from pycox.utils import kaplan_meier
import logging
import logging.config
import pytz
import torch
import plotly.graph_objs as go
# from lifelines import KaplanMeierFitter
import plotly
plotly.offline.init_notebook_mode(connected=True)
import pandas as pd
%load_ext autoreload
%autoreload 2
import time
from torch import nn
import nibabel as nib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create Dataset

In [6]:
preop_patients = []
for path in Path('./data/preoperative_no_norm').glob('BMIAXNA*'):
    preop_patients.append(path)

In [7]:
id_mapping = './data/pickles_jsons/id_surv_mapping_10_groups.json'

In [8]:
mapper_class = fm(preop_patients, id_mapping, normalized=True)
dataset = mapper_class.generate_mapping()

Number of patients/folders: 1016


In [9]:
with open('./data/pickles_jsons/filter_ids_v2_all.pkl', 'rb') as file:
    filter_ids = pickle.load(file)
dataset_filtered = [entry for entry in dataset if entry['ENT'] is not None]
dataset_filtered = [entry for entry in dataset_filtered if entry['id'] not in filter_ids]

## Set Random seed and randomize dataset

In [10]:
random.seed(4)
random.shuffle(dataset_filtered)

# Test dataset

In [12]:
test_dataset = dataset_2(dataset_filtered, phase='test', normalize='zscore', size=64)

Test Size: 204


# DataLoaders

In [13]:
test_loader = DataLoader(test_dataset,
                         batch_size=204,
                         shuffle=False)

# ResNet-10

In [14]:
cuts = np.array([   0.,   66.,  141.,  208.,  292.,  363.,  449.,  592.,  829., 1785.])

In [15]:
net = resnet.generate_model(model_depth=10,
                        n_classes=10,
                        n_input_channels=4,
                        shortcut_type='B',
                        conv1_t_size=7,
                        conv1_t_stride=1,
                        no_max_pool=False,
                        widen_factor=1.0)
# net.to(torch.device("cuda:0"))

In [16]:
net.to(torch.device("cuda:0"))

ResNet(
  (conv1): Conv3d(4, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=T

In [17]:
net.load_state_dict(torch.load('./best_model.pt'))

<All keys matched successfully>

# Extract 3D Convolutional Layers

In [19]:
model_weights = [] # save the conv layer weights in this list
conv_layers = [] # save the 9 conv layers in this list
# get all the model children as list
model_children = list(net.children())

In [22]:
# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv3d:
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv3d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolutional layers: {counter}")

Total convolutional layers: 9


In [222]:
outputs[3].shape

torch.Size([64, 64, 32, 32])

In [23]:
for weight, conv in zip(model_weights, conv_layers):
    # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")

CONV: Conv3d(4, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False) ====> SHAPE: torch.Size([64, 4, 7, 7, 7])
CONV: Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False) ====> SHAPE: torch.Size([64, 64, 3, 3, 3])
CONV: Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False) ====> SHAPE: torch.Size([64, 64, 3, 3, 3])
CONV: Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False) ====> SHAPE: torch.Size([128, 64, 3, 3, 3])
CONV: Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False) ====> SHAPE: torch.Size([128, 128, 3, 3, 3])
CONV: Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False) ====> SHAPE: torch.Size([256, 128, 3, 3, 3])
CONV: Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False) ====> SHAPE: torch.Size([256, 256, 3, 3, 3])
CONV: Conv3d(256, 512, kernel_size=(

# Create custom convolutional layers and plot for 1 patient sequence filters

## T1c

In [192]:
conv1 = nn.Conv3d(1, 64, kernel_size=(7,7,7), stride=(1,2,2), padding=(3,3,3),  bias=False)

In [193]:
conv1.weight = torch.nn.Parameter(model_children[0].weight[:,0,:,:].unsqueeze(1))

In [195]:
x = img[0].unsqueeze(0).unsqueeze(0)

In [198]:
out = conv1(x)

In [199]:
out.size()

torch.Size([1, 64, 64, 32, 32])

In [231]:
out[0][2].size()

torch.Size([64, 32, 32])

In [ ]:
returnMontage(out[0][0].detach().cpu().numpy(), 'T1c')

# FLR

In [207]:
conv1_flr = nn.Conv3d(1, 64, kernel_size=(7,7,7), stride=(1,2,2), padding=(3,3,3),  bias=False)

In [216]:
conv1_flr

Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)

In [228]:
model_children[0].weight[:,2,:,:].size()

torch.Size([64, 7, 7, 7])

In [208]:
conv1_flr.weight = torch.nn.Parameter(model_children[0].weight[:,2,:,:].unsqueeze(1))

In [209]:
x_flr = img[2].unsqueeze(0).unsqueeze(0)

In [217]:
x_flr.size()

torch.Size([1, 1, 64, 64, 64])

In [210]:
out_flr = conv1_flr(x_flr)

In [212]:
out_flr.size()

torch.Size([1, 64, 64, 32, 32])

In [213]:
out_flr[0][0].size()

torch.Size([64, 32, 32])

In [ ]:
returnMontage(out_flr[0][3].detach().cpu().numpy(), 'FLR')

# T1c second layer

In [272]:
conv1_t1c_2 = nn.Conv3d(1, 64, kernel_size=(7,7,7), stride=(1,2,2), padding=(3,3,3),  bias=False)

In [273]:
conv1_t1c_2

Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)

In [282]:
model_weights[1][:,0,:,:].size()

torch.Size([64, 3, 3, 3])

In [284]:
conv1_t1c_2.weight = torch.nn.Parameter(model_weights[1][:,0,:,:].unsqueeze(1))

In [285]:
x_t1c_2 = img[0].unsqueeze(0).unsqueeze(0)

In [286]:
x_t1c_2.size()

torch.Size([1, 1, 64, 64, 64])

In [287]:
out_t1c_2 = conv1_t1c_2(x_t1c_2)

In [288]:
out_t1c_2.size()

torch.Size([1, 64, 68, 34, 34])

In [289]:
out_t1c_2[0][0].size()

torch.Size([68, 34, 34])

In [ ]:
returnMontage(out_t1c_2[0][3].detach().cpu().numpy(), 'FLR')

# Get One test subject

In [28]:
img = test_dataset[5].cuda()

In [29]:
img.shape

torch.Size([4, 64, 64, 64])

# Get Full Feature maps of test subject

In [299]:
results = [conv_layers[0](img.unsqueeze(0))]

In [107]:
results[0][0].shape

torch.Size([64, 64, 32, 32])

In [300]:
for i in range(1, len(conv_layers)):
    # pass the result from the last layer to the next layer
    results.append(conv_layers[i](results[-1]))
# make a copy of the `results`
outputs = results

## First feature map (first conv layer, filter 2)

In [310]:
layer_viz = outputs[0][0, 2, :, :, :]
layer_viz = layer_viz.data
print(layer_viz.size())

torch.Size([64, 32, 32])


## Second feature map (fifth conv layer, filter 50)

In [84]:
layer_viz_2 = outputs[5][0, 50, :, :, :]
layer_viz_2 = layer_viz_2.data
print(layer_viz_2.size())

torch.Size([16, 8, 8])


In [43]:
x= np.random.random((3,3, 3))

In [44]:
x.shape

(3, 3, 3)

In [45]:
layer_viz_2.shape

torch.Size([64, 32, 32])

### Normalize -1 to 1 range (not really needed)

In [311]:
# Amax = np.max(layer_viz.cpu().numpy())
# Amin = np.min(layer_viz.cpu().numpy())
# Range = Amax - Amin
# Anrm = ((layer_viz - Amin)/Range - 0.5) * 2   

In [1]:
# np.min(Anrm.cpu().numpy())

In [60]:
# ni_img = nib.Nifti1Image(Anrm.cpu().numpy(), np.eye(4))
# nib.save(ni_img, 'normalized_featuremap.nii.gz')

# Plot slices

In [ ]:
returnMontage(test_dataset[5][0], 'Actual')

In [ ]:
returnMontage(layer_viz.detach().cpu(), 'Feature map 1')

In [ ]:
returnMontage(layer_viz_2.detach().cpu(), 'Feature map 2')